In [1]:
from pathlib import Path
import random
import numpy as np
import scipy.stats
import matplotlib.pyplot as plt
from pyfaidx import Fasta
from meth5 import MetH5File
import pandas as pd

from bayes_opt import BayesianOptimization
from pomegranate import NormalDistribution, BetaDistribution, GeneralMixtureModel

from nanoepitools.plotting.general_plotting import PlotArchiver, plot_2d_density
from nanoepitools.math import llr_to_p, p_to_llr
from benchmark_pycometh.config import module_config
from benchmark_pycometh.simulation.nanopolish_simulator import OmicsSimlaLoader

In [ ]:
def estimate_beta_dist(x):
    mean = np.mean(x)
    var = np.var(x)
    alpha = ((1-mean) / var - 1/mean) *  mean**2
    beta = alpha * (1/mean - 1)
    return alpha, beta

In [ ]:
pa = PlotArchiver("simulation", headless=False, config={"plot_archive_dir": "/home/r933r/snajder/nanoepitools_plots/benchmark"})

Fasta(module_config.reference)

In [ ]:
mf = MetH5File(module_config.meth5_template_file.format(sample="HG002"), "r")

In [ ]:
llrs = mf["21"].get_all_values().get_llrs()

pa.figure()
plt.hist(np.clip(llrs, -20, 20), bins=40)
plt.xlim(-20,20)
plt.show()

In [ ]:
ps = llr_to_p(llrs)

pa.figure()
plt.hist(ps, bins=40)
plt.xlim(0,1)
plt.show()

In [ ]:
pa.figure()
plt.hist(np.abs(ps*2-1), bins=40)
plt.xlim(0,1)
plt.show()

In [ ]:
s = np.random.beta(*estimate_beta_dist(np.abs(ps*2-1)), 10000)
pa.figure()
plt.hist(s, bins=40)
plt.xlim(0,1)
plt.show()

In [ ]:
estimate_beta_dist(np.abs(ps*2-1))

In [ ]:


ps_clipped = np.clip(ps, 1e-10, 1-1e-10)
def loss_bimodal(a, b, mu, sigma, w1):
    beta_loss = BetaDistribution(a, b).probability(ps_clipped)
    norm_loss = NormalDistribution(mu, sigma).probability(ps_clipped)
    return np.log(w1 * beta_loss + (1 - w1) * norm_loss).sum()

def pdf_bimodal(a, b, mu, sigma, w1, x=np.arange(0, 1, 0.01)):
    return w1 * BetaDistribution(a, b).probability(x) + \
        (1 - w1) * NormalDistribution(mu, sigma).probability(x)

optimizer = BayesianOptimization(
    f=loss_bimodal,
    pbounds={'mu': (0.25, 0.75),
             'sigma': (0.01, 0.1),
             'a': (0.01, 0.99),
             'b': (0.01, 0.99),
             'w1': (0.8, 0.99)},
    random_state=1
)
optimizer.maximize(
    init_points=5,
    n_iter=100, kappa_decay=0.95
)

In [ ]:
a, b, mu, sigma, w1 = [v for k, v in optimizer.max['params'].items()]
x = np.arange(0, 1, 0.001)
pa.figure()
plt.plot(x, pdf_bimodal(a, b, mu, sigma, w1, x))
plt.hist(ps_clipped, density=True, bins=100)
plt.show()

In [ ]:
model = GeneralMixtureModel([NormalDistribution(mu, sigma), BetaDistribution(a,b)], weights=[1-w1, w1])
s = model.sample(10000)
pa.figure()
plt.hist(s, bins=40)
plt.xlim(0,1)
plt.show()

In [ ]:
bs = mf["1"].get_all_values().get_llr_site_readgroup_rate("haplotype")

In [ ]:
pa.figure()
plt.hist(np.concatenate((bs[1][0],bs[2][0],bs[3][0],bs[0][0])), bins=40)
plt.show()

In [ ]:
bs = mf["1"].get_all_values().get_llr_site_rate()
pa.figure()
plt.hist(bs[0], bins=40)
plt.show()

In [ ]:
bs_a, bs_b = estimate_beta_dist(bs[0])

s = BetaDistribution(bs_a, bs_b).sample(len(bs[0]))
pa.figure()
plt.hist(s, bins=40)
plt.hist()

In [ ]:
omics_simla_dir = Path("/home/r933r/data/projects/nanopore/pycometh_benchmark/simulated/wgbs/unmetbig")
omics_simla_profile_path = Path("/home/r933r/data/software/users/snajder/OmicsSIMLA_v0.6/profiles/methylation/WGBS/liver_profile.txt")
omics_simla_profile_map_path = Path("/home/r933r/data/software/users/snajder/OmicsSIMLA_v0.6/profiles/methylation/WGBS/map.txt")

sim = OmicsSimlaLoader(omics_simla_dir, omics_simla_profile_path, omics_simla_profile_map_path)



In [ ]:
for sample in sim.rates:
    sim.summary[sample] = sim.rates[sample]

sim.summary["segment_rate"] = sim.profile_rates

In [ ]:

pa.figure()
plot_2d_density(np.array(sim.profile_rates), np.array(sim.summary["DCONTROLS1"]), cmap="jet")
plt.gca().set_aspect("equal")
plt.plot([0,1],[0,1])
plt.show()

pa.figure()
plot_2d_density(np.array(sim.profile_rates), np.array(sim.summary["DCASES1"]), cmap="jet")
plt.gca().set_aspect("equal")
plt.plot([0,1],[0,1])
plt.show()

In [ ]:
idx = np.array(sim.summary["Theta"] < 0) & (sim.segment_types == 1)
pa.figure()
plot_2d_density(np.array(sim.summary["DCONTROLS1"])[idx], np.array(sim.summary["DCASES1"])[idx], cmap="jet", nbins=100)
plt.gca().set_aspect("equal")
plt.plot([0,1],[0,1])
plt.show()

pa.figure()
plot_2d_density(sim.profile_rates[idx], np.array(sim.summary["DCASES1"])[idx], cmap="jet", nbins=100)
plt.gca().set_aspect("equal")
plt.plot([0,1],[0,1])
plt.show()

In [ ]:
def read_fq_read_lengths(fq_files, minlen=100):
    for fq_file in fq_files:
        with open(fq_file, "r") as f:
            while True:
                try:
                    _ = next(f) # header
                    seq = next(f) # seq
                    _ = next(f) # header
                    _ = next(f) # qual
                    l = len(seq)
                    if l > minlen:
                        yield l
                except StopIteration:
                    break

fq_dir = "/home/r933r/data/projects/nanopore/pycometh_benchmark/fastq/HG002/"
batches = [f"{fq_dir}/{i}.fq" for i in range(20)]
read_lens = list(read_fq_read_lengths(batches))

In [ ]:
pa.figure()
plt.hist(np.log10(read_lens), bins=100)
plt.show()

In [ ]:
readlen_model = GeneralMixtureModel([NormalDistribution(3,0.5), NormalDistribution(4,0.5), NormalDistribution(5,0.5)])
readlen_model.fit(np.log10(read_lens))

In [ ]:
s = readlen_model.sample(len(read_lens)//50)
pa.figure()
plt.hist(s, bins=100, alpha=0.5, label="sim")
#plt.hist(np.log10(read_lens), bins=100, alpha=0.5, label="real")
plt.legend()
plt.show()

In [2]:
99 999 954 / 500 / 100

1999.99908